In [320]:
in_colab = 'google.colab' in str(get_ipython())
if in_colab == True:
  !pip install yfinance
  import hvplot.pandas
  import plotly.express as px
  import yfinance as yf
  import numpy as np
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from sklearn.metrics import accuracy_score
  import matplotlib.pyplot as plt
  from sklearn.kernel_ridge import KernelRidge
  import numpy as np
  from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
else:
  import hvplot.pandas
  import plotly.express as px
  import yfinance as yf
  import numpy as np
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from sklearn.metrics import accuracy_score
  import matplotlib.pyplot as plt
  from sklearn.kernel_ridge import KernelRidge
  import numpy as np
  from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score

In [321]:
crypto_btc = yf.download(tickers='BTC-USD', start='2017-12-01', end='2022-12-31').drop(columns=['Adj Close']).round(2)

[*********************100%***********************]  1 of 1 completed


In [322]:
crypto_btc['EMA_9'] = crypto_btc['Close'].ewm(9).mean().shift()
crypto_btc['SMA_5'] = crypto_btc['Close'].rolling(5).mean().shift()
crypto_btc['SMA_10'] = crypto_btc['Close'].rolling(10).mean().shift()
crypto_btc['SMA_15'] = crypto_btc['Close'].rolling(15).mean().shift()
crypto_btc['SMA_30'] = crypto_btc['Close'].rolling(30).mean().shift()
crypto_btc['Percent Change'] = crypto_btc['Close'].pct_change()

In [323]:
crypto_btc = crypto_btc.dropna()
crypto_btc

,Open,High,Low,Close,Volume,EMA_9,SMA_5,SMA_10,SMA_15,SMA_30,Percent Change
Date,,,,,,,,,,,
2017-12-31 00:00:00+00:00,12897.70,14377.40,12755.60,14156.40,12136299520,15307.711376,14830.640,14643.950,15906.213333,15332.200000,0.092973
2018-01-01 00:00:00+00:00,14112.20,14112.20,13154.70,13657.20,10291200000,15188.013521,14441.960,14479.300,15550.146667,15438.226667,-0.035263
2018-01-02 00:00:00+00:00,13625.00,15444.60,13163.60,14982.10,16846600192,15029.488936,14005.700,14461.840,15184.573333,15524.313333,0.097011
2018-01-03 00:00:00+00:00,14978.20,15572.80,14844.50,15201.00,16871900160,15024.598925,14080.820,14490.130,14909.100000,15646.276667,0.014611
2018-01-04 00:00:00+00:00,15270.70,15739.70,14522.20,15599.20,21783199744,15042.743690,14189.780,14617.650,14737.386667,15764.403333,0.026196
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26 00:00:00+00:00,16842.25,16920.12,16812.37,16919.80,11886957804,16930.292092,16826.916,16768.111,16996.905333,16939.856000,0.004620
2022-12-27 00:00:00+00:00,16919.29,16959.85,16642.07,16717.17,15748580239,16929.242883,16847.368,16795.343,16984.612667,16955.040000,-0.011976
2022-12-28 00:00:00+00:00,16716.40,16768.17,16497.56,16552.57,17005713920,16908.035594,16824.734,16787.551,16951.994667,16964.124667,-0.009846


In [324]:
crypto_btc.reset_index(inplace=True)
fig = px.line(crypto_btc, x='Date', y='Close', title='Close Price with Range Slider')
fig.update_layout(hovermode="x")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=2, label="2y", step="year", stepmode="backward"),
            dict(step="all")
            
        ])
    )
)
fig.show()

In [325]:
'''
df = ()
def kernel_predict(df_pred):
    X = df.drop(columns=['Date'])
    y = df[['Close']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
    if df is crypto_btc:
        kr = KernelRidge(kernel='linear', alpha=5000)
    else:
        kr = KernelRidge(kernel='polynomial', degree=1, gamma=.05)
    kr.fit(X_train, y_train)
    predict = kr.predict(X)
    if df is crypto_btc:
        df_pred = df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
    else:
        df_pred = df.copy().drop(columns=['Open']).rename(columns={'Close':'Actual Price'})
    df_pred['Predicted Price'] = predict
    df_pred['Price Accuracy']= (df_pred['Predicted Price']/df_pred['Actual Price']-1)*100
    return df_pred
'''

"\ndf = ()\ndef kernel_predict(df_pred):\n    X = df.drop(columns=['Date'])\n    y = df[['Close']]\n    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)\n    if df is crypto_btc:\n        kr = KernelRidge(kernel='linear', alpha=5000)\n    else:\n        kr = KernelRidge(kernel='polynomial', degree=1, gamma=.05)\n    kr.fit(X_train, y_train)\n    predict = kr.predict(X)\n    if df is crypto_btc:\n        df_pred = df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})\n    else:\n        df_pred = df.copy().drop(columns=['Open']).rename(columns={'Close':'Actual Price'})\n    df_pred['Predicted Price'] = predict\n    df_pred['Price Accuracy']= (df_pred['Predicted Price']/df_pred['Actual Price']-1)*100\n    return df_pred\n"

In [326]:
X = crypto_btc.drop(columns=['Date'])
y = crypto_btc[['Close']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
kr = KernelRidge(kernel='linear', alpha=5000)
kr.fit(X_train, y_train)
predict = kr.predict(crypto_btc.drop(columns=['Date']))
crypto_btc_pred = crypto_btc.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
crypto_btc_pred['Predicted Price'] = predict
crypto_btc_pred['Price Accuracy']= (crypto_btc_pred['Predicted Price']/crypto_btc_pred['Actual Price']-1)*100
    

/Users/playb3yond40gb/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:196: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.



In [327]:
btc_mae, btc_r_square, btc_r_square= (),(),()

btc_mae = mean_absolute_error(y['Close'], predict)
btc_rmse = np.sqrt(mean_squared_error(y['Close'], predict))
btc_r_square = r2_score(y['Close'], predict)
print(btc_mae)
print(btc_rmse)
print(btc_r_square)

138.17981927710844
217.04240415471716
0.9998366480654297


In [328]:
crypto_btc_pred

,Date,High,Low,Actual Price,EMA_9,SMA_5,SMA_10,SMA_15,SMA_30,Percent Change,Predicted Price,Price Accuracy
0,2017-12-31 00:00:00+00:00,14377.40,12755.60,14156.40,15307.711376,14830.640,14643.950,15906.213333,15332.200000,0.092973,14022.0,-0.949394
1,2018-01-01 00:00:00+00:00,14112.20,13154.70,13657.20,15188.013521,14441.960,14479.300,15550.146667,15438.226667,-0.035263,13635.0,-0.162552
2,2018-01-02 00:00:00+00:00,15444.60,13163.60,14982.10,15029.488936,14005.700,14461.840,15184.573333,15524.313333,0.097011,14950.0,-0.214256
3,2018-01-03 00:00:00+00:00,15572.80,14844.50,15201.00,15024.598925,14080.820,14490.130,14909.100000,15646.276667,0.014611,15402.0,1.322281
4,2018-01-04 00:00:00+00:00,15739.70,14522.20,15599.20,15042.743690,14189.780,14617.650,14737.386667,15764.403333,0.026196,15580.0,-0.123083
...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2022-12-26 00:00:00+00:00,16920.12,16812.37,16919.80,16930.292092,16826.916,16768.111,16996.905333,16939.856000,0.004620,16922.0,0.013003
1822,2022-12-27 00:00:00+00:00,16959.85,16642.07,16717.17,16929.242883,16847.368,16795.343,16984.612667,16955.040000,-0.011976,16762.0,0.268167
1823,2022-12-28 00:00:00+00:00,16768.17,16497.56,16552.57,16908.035594,16824.734,16787.551,16951.994667,16964.124667,-0.009846,16600.0,0.286542
1824,2022-12-29 00:00:00+00:00,16651.76,16508.68,16642.34,16872.489035,16775.858,16767.010,16870.078000,16975.299667,0.005423,16626.0,-0.098183


In [329]:
crypto_btc_pred['Price Accuracy'].describe()

count    1826.000000
mean        0.600491
std         0.937569
min        -3.327315
25%         0.035102
50%         0.499143
75%         1.000843
max        13.301910
Name: Price Accuracy, dtype: float64

In [330]:
crypto_btc_pred.hvplot(
    x='Date',
    y=[('Actual Price') ,'Predicted Price'],
    grid=True
)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [331]:
print(f"BTC Kernel Ridge RMSE is {btc_rmse.round(2)}")

BTC Kernel Ridge RMSE is 217.04


In [345]:
future_df = pd.DataFrame()
  

future_df['Date'] = pd.date_range('2018-02-01', periods = 1826, freq ='1D')
future_df[['Close','Open','Volume']]= (0)

future_df


,Date,Close,Open,Volume
0,2018-02-01,0,0,0
1,2018-02-02,0,0,0
2,2018-02-03,0,0,0
3,2018-02-04,0,0,0
4,2018-02-05,0,0,0
...,...,...,...,...
1821,2023-01-27,0,0,0
1822,2023-01-28,0,0,0
1823,2023-01-29,0,0,0
1824,2023-01-30,0,0,0


In [346]:
crypto_btc
X = crypto_btc.drop(columns=['Date'])
y = crypto_btc[['Close']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
kr = KernelRidge(kernel='linear', alpha=5000)
kr.fit(X_train, y_train)
predict = kr.predict(crypto_btc.drop(columns=['Date']))
btc_future = future_df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
btc_future['Predicted Price'] = predict

btc_future 

/Users/playb3yond40gb/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:196: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.



,Date,Actual Price,Predicted Price
0,2018-02-01,0,14022.0
1,2018-02-02,0,13635.0
2,2018-02-03,0,14950.0
3,2018-02-04,0,15402.0
4,2018-02-05,0,15580.0
...,...,...,...
1821,2023-01-27,0,16922.0
1822,2023-01-28,0,16762.0
1823,2023-01-29,0,16600.0
1824,2023-01-30,0,16626.0


In [347]:
btc_mae, btc_r_square, btc_r_square= (),(),()

btc_mae = mean_absolute_error(y['Close'], predict)
btc_rmse = np.sqrt(mean_squared_error(y['Close'], predict))
btc_r_square = r2_score(y['Close'], predict)
print(btc_mae)
print(btc_rmse)
print(btc_r_square)

138.17981927710844
217.04240415471716
0.9998366480654297


In [351]:
crypto_btc2 = yf.download(tickers='BTC-USD', start='2018-01-01', end='2023-01-30').drop(columns=['Adj Close']).round(2).reset_index()
crypto_btc2['EMA_9'] = crypto_btc2['Close'].ewm(9).mean().shift()
crypto_btc2['SMA_5'] = crypto_btc2['Close'].rolling(5).mean().shift()
crypto_btc2['SMA_10'] = crypto_btc2['Close'].rolling(10).mean().shift()
crypto_btc2['SMA_15'] = crypto_btc2['Close'].rolling(15).mean().shift()
crypto_btc2['SMA_30'] = crypto_btc2['Close'].rolling(30).mean().shift()
crypto_btc2['Percent Change'] = crypto_btc2['Close'].pct_change()
#crypto_btc2 = crypto_btc2.dropna()
crypto_btc2
btc_future['Actual Price'] = crypto_btc2['Close'].round(0)
btc_future['Price Accuracy']= (btc_future['Predicted Price']/btc_future['Actual Price']-1)*100
btc_future.set_index('Date').tail(30)

[*********************100%***********************]  1 of 1 completed


,Actual Price,Predicted Price,Price Accuracy
Date,,,
2023-01-02,17089.0,17064.0,-0.146293
2023-01-03,16908.0,17144.0,1.395789
2023-01-04,17130.0,17000.0,-0.758903
2023-01-05,16975.0,17154.0,1.054492
2023-01-06,17090.0,17122.0,0.187244
2023-01-07,16848.0,17096.0,1.471985
2023-01-08,17233.0,16888.0,-2.001973
2023-01-09,17133.0,17218.0,0.496119
2023-01-10,17129.0,17188.0,0.344445


In [349]:
display(btc_future.loc[btc_future['Date'] == '2020-01-27'])
display(btc_future.loc[btc_future['Date'] == '2020-02-27'])

,Date,Actual Price,Predicted Price,Price Accuracy
725,2020-01-27,7290.0,7322.0,0.438957


,Date,Actual Price,Predicted Price,Price Accuracy
756,2020-02-27,8910.0,8596.0,-3.52413


In [350]:
from bokeh.models.formatters import BasicTickFormatter

btc_future.hvplot(
    x='Date',
    y=[('Actual Price') ,'Predicted Price'],
    grid=True
)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)